# Pandas 及应用

Pandas是数据分析最常用的包:

- Pandas 定义了处理数据的结构；
- 数据处理：读取、调整指数、日期和时间序列、排序、分组、处理缺失值；
- 一些更复杂的统计功能，如 statsmodels 和 scikit-learn，也是建立在pandas基础上。

## Pandas 序列



Pandas中两类数据，Series 和 DataFrame；

Series 基于Numpy数组，支持许多类似运算；

Series 可以看作一“列”数据；

DataFrame 可以看作储存相应列数据的二维对象；类似Excel表单；

Series一些方法


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(123)
s = pd.Series(np.random.randn(100), name="daily return")
s.plot();
np.abs(s)
s.describe()

### Pandas数据框


DataFrames 是几列数据组成，每一列对应一个变量；

用来方便的处理行和列组织的数据；索引（index）对应行，变量列名（columns）对应列；

可以读取各类软件格式存储数据，csv, excel, stata, html, json,sql等；


## 应用：Penn World Table

这一部分应用[Penn World Table](https://www.rug.nl/ggdc/productivity/pwt/)介绍对原始数据的一些常见处理方法。该数据集当前版本为PWT 10.01，包含183个国家1950-2019年的收入、产出、投入和生产率等指标，详细介绍可参见[User Guide to PWT 10.0 data files](https://www.rug.nl/ggdc/docs/pwt100-user-guide-to-data-files.pdf)。数据背后的方法、理论及使用建议，可参见 @feenstra2015next。

网站提供了Stata和Excel格式数据，这里我们下载了后者。数据本身是一个面板数据（Panel Data），“国家 - 年” 唯一识别一个观测值。我们从截面数据入手先只保留2019年数据， 然后再看更复杂的情况。

### 导入数据

假设数据保存在当前路径的datasets子文件中：


In [ ]:
import pandas as pd
pwt = pd.read_excel(io = "datasets/pwt1001.xlsx",
                header=0,                
                sheet_name="Data")
# 保留2019年数据
pwt2019 = pwt[pwt['year'] == 2019].copy().drop(labels='cor_exp',axis=1)

注意其中的几个参数，`io`是文件路径；`header`表明列标题行，这里是第一行；`sheet_name`是数据所在表单名；将载入的数据赋值给pwt数据框。我们只保留2019年的观测值，变量`cor_exp`在这一年全部为缺失值，这里直接删除了。

先为`pwt2019`数据框设置索引变量，这里使用国家名代码变量（countrycode）：


In [ ]:
pwt2019.set_index('countrycode', inplace=True)

可以`df.info()`概率数据集，或者使用`df.head()`或`df.tail()`查看头部和尾部观测值：


In [ ]:
pwt2019.info()
pwt2019.head()

默认显示5条观测值，如果希望看到更多观测值，可以使用 `df.tail(n=10)` 修改数值。

可以应用`.shape, .ndim`,`.columns`等属性查看基本信息，可以看到数据集包含51个变量共183个观测值。


In [ ]:
print(pwt2019.shape)
print(pwt2019.columns)

### 选择观测值和变量

应用中经常对某些观测值或特定子集进行操作，因此很重要的一步是选择观测值和变量。

最基本的方法可以通过Python数组的切片（slicing）方式选择特定的**行**。例如，选择第3至5个观测值：


In [ ]:
pwt2019[2:5]

要选择**列**，可以用包含列名字的列表：


In [ ]:
vars_selected = ['country', 'rgdpe', 'rgdpo', 'pop', 'emp', 'cgdpe', 'cgdpo', 'ctfp' ]
df = pwt2019[vars_selected]

#### `.loc`方法

`.loc` 是基于 标签（label-based） 的数据选择方法。这意味着你使用行和列的实际标签名来选择数据，而不是它们的整数位置。

例如，要选择金砖国家（BRICKS）的观测值：


In [ ]:
bricks = ['CHN', 'BRA', 'RUS', 'IND', 'ZAF']
pwt2019.loc[bricks]

或者选择列：


In [ ]:
variables = ['country', 'rgdpe', 'pop']
pwt2019.loc[:, variables]

或者同时指定行和列：


In [ ]:
pwt2019.loc[bricks, variables]

#### `.iloc`方法

相应的，`.iloc` 是基于整数位置（integer-location based）的，使用行和列的整数位置（从 0 开始）来选择数据。例如：


In [ ]:
# 选择第2行数据（索引位置为1）
pwt2019.iloc[1]
# 选择第1行（索引为0）、第3行（索引为2）和第5行（索引为4）
pwt2019.iloc[[0, 2, 4]]
# 选择前5行、第4至第6列观测值
pwt2019.iloc[:5, 3:6]

这里需要注意Python中索引位置。Python中进行切片（slicing）操作时，语法通常类似 `[start:end]`，要注意：

- `start`：切片的起始索引，对应的元素会被包含。
- `end`：切片的结束索引，对应的元素不会被包含。
  
#### 根据条件筛选

除了根据索引或位置选择数据外，也可以利用条件来筛选观测值。例如，根据人口变量（`pop`，单位：百万）选择2019年总人口超过2亿的观测值：


In [ ]:
pwt2019[pwt2019['pop'] >= 200]

注意，`pwt2019['pop'] >= 200` 的结果是一列布林值，然后`pwt2019[]`选择返回取值为`True`的观测值。

再例如，下面的代码包含了两个条件：

- 国家名属于金砖国家。注意这里使用了Pandas 中的`df.isin()`函数；
- 2019年人口超过10亿。
  
当有不止一个条件时，我们用`&`, `|`表示`and` 和 `or`运算符；


In [ ]:
BRICKS = ['China','Brazil',  ' Russian Federation', 'India', 'South Africa']
#
pwt2019[(pwt2019['country'].isin(BRICKS)) & (pwt2019['pop'] > 1000)]

更复杂的情况，可以在条件语句中加入数学表达式。例如，下面的代码筛选了人均实际GDP超过2万美元和人口超过5000万的国家的观测值，这里人均实际GDP是购买力平价调整后支出法衡量的实际GDP与人口的比值：


In [ ]:
pwt2019[(pwt2019['rgdpe']/pwt2019['pop'] > 20000) & (pwt2019['pop'] > 50)]

### apply 方法

Pandas中一个广泛应用的方法是 `df.apply()`，它将一个函数应用到每一行/列，返回一个序列；

函数可以是内嵌的（built in）也可以是自定义的，例如，计算每一列的最大值，为了节省输出空间，使用子集`df`数据框：


In [ ]:
df.apply(np.max, axis=0)

或者，自定义一个函数`range(x)`计算极差：


In [ ]:
import numpy as np
def range(x):
    return np.max(x) - np.min(x)
df.select_dtypes(np.number).apply(range)

再例如，归一化（normalization）经常使用minmax方法：
$$
Y = \frac{X_{i} - \min(X_{i})}{\max(X_{i}) - \min(X_{i})}
$$

我们定义一个函数`minmax()`，然后应用`apply()`方法：


In [ ]:
def minmax(S):
    return (S-S.min())/(S.max() - S.min())
pwt2019[['pop','rgdpe', 'emp']].apply(minmax)

经常将`lambda`函数方法与`df.apply()`方法相结合。例如，数据集中有4个指标度量GDP，分别是`['rgdpe', 'rgdpo','cgdpe','cgdpo']`，假设我们希望计算一个加权平均数，权重为（0.3，0.2，0.3，0.2）：


In [ ]:
variables = ['rgdpe', 'rgdpo','cgdpe','cgdpo']
df[variables].apply(lambda row:
    row['rgdpe']*0.3 + row['rgdpo']*0.2 + row['cgdpe']*0.3 + row['cgdpo']*0.2,
    axis=1)

注意，z选项`axis = 1` ，将函数应用至每一行，默认值为0。


### 检测和处理缺失值

Pandas中最常用的缺失值表示是`NaN`（Not a Number）。可以使用`isnull()`或`isna()`函数检测缺失值，返回一个布尔型的DataFrame，其中`True`表示缺失值：


In [ ]:
pwt2019.isna()
#pwt2019.isnull()

下面的的代码计算了缺失值的数量，将其除以样本容量得到缺失值比例，然后按照降序排序，并将比例最高的前15个变量绘制柱形图：


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
(pwt2019.isna().sum()/pwt2019.shape[0]*100).sort_values(ascending=False)[:15].plot(kind='bar', ax=ax)
ax.set_ylabel("%")
plt.show()

另一种图示的方法是类似矩阵绘图的方式，将缺失值标记出来，`missingno`库有简单的命令实现：


In [ ]:
import missingno as msno
plt.figure(figsize=(12, 6))
msno.matrix(pwt2019)
plt.title("Missing Values Matrix")
plt.show()

**删除缺失值**

处理缺失值的方法有很多种，选择哪种方法取决于你的数据特性、缺失原因以及分析目标。最直接的方法是使用`df.dropna()`函数删除包含缺失值的行或列：


In [ ]:
# 删除含缺失值的行
pwt2019.dropna()
# 删除含缺失值的列
pwt2019.dropna(axis=1)

另外，上面的命令并没有改变原数据框，可以通过赋值方式保存。或者加上选项`df.dropna(inplace=True)`，即在原数据框中生效。

**填充**

`df.fillna()`是用于填充缺失值的核心函数。


In [ ]:
#
pwt2019.fillna(0)
#
pwt2019.select_dtypes(np.number).fillna(0).combine_first(pwt2019)
pwt2019.select_dtypes(np.number).fillna(pwt2019.mean(numeric_only=True)).combine_first(pwt2019)
pwt2019.select_dtypes(np.number).fillna(pwt2019.median(numeric_only=True)).combine_first(pwt2019)

In [ ]:
#pwt2019.fillna(method='ffill')
pwt2019.fillna(method='bfill')

**插值法（Interpolation）**

除了填充给定值以外，也有更复杂的插值法。


In [ ]:
pwt2019.interpolate(method="linear")

更复杂的方法涉及到模型估计问题，如KNN预测等。Scikit-learn库有专门的方法，这里就不多涉及。


In [ ]:
from sklearn.impute import SimpleImputer
imputer_mean = SimpleImputer(strategy='mean')
pd.DataFrame(imputer_mean.fit_transform(pwt2019.select_dtypes(np.number)), columns=pwt2019.select_dtypes(np.number).columns)

### 缩尾处理

应用中，常需要对异常值进行一定的处理，其中一种方法是缩尾处理（Winsorize），将极端值替换为某个百分位数的值，例如，将上限设为 99 百分位数，下限设为 1 百分位数。

可以使用`df.clip()`函数实现，例如全要素生产率水平`ctfp`：


In [ ]:
q95 = pwt2019['ctfp'].quantile(0.95)
q05 = pwt2019['ctfp'].quantile(0.05)

pwt2019['ctfp'].dropna().clip(lower=q05, upper=q95, inplace=False)

### 观测值排序

有时候需要对数据集进行一定的排序，Pandas中可以按索引(`df.sort_index`)和值（`df.sort_values`）排序。

例如，将索引按降序排序，这里的索引是国家代码，因此升序/降序是按照字母顺序：


In [ ]:
pwt2019.sort_index(ascending=False)

来看`df.sort_values`的例子，假设我们希望按2019年的人均GDP（PPP链式调整后）降序排列：


In [ ]:
pwt2019['rgdp_per'] = pwt2019['rgdpe']/pwt2019['pop']
pwt2019.sort_values(by='rgdp_per', ascending=False) 

### 数据集合并

实际应用中，数据可能来自不同的来源，经常需要合并数据集，`pd.merge()`函数


In [ ]:
import wbgapi as wb
inf = wb.data.DataFrame(series='NY.GDP.DEFL.KD.ZG', time='2019')
pd.merge(df[['country','pop','emp']], inf, left_index=True, right_index=True)

### 多级索引

这里的数据是一个面板数据，“国家-年”对应一个观测值，可以利用Pandas的多级索引功能，详见Pandas文档[MultiIndex / advanced indexing](https://pandas.pydata.org/docs/dev/user_guide/advanced.html#)。


In [ ]:
pwt = pd.read_excel(io = "datasets/pwt1001.xlsx",
                header=0,                
                sheet_name="Data")
pwt.set_index(['countrycode','year'], inplace=True)

我们可以使用`.loc()`方法选择需要的数据，例如：


In [ ]:
# 中国子集
df_china = pwt.loc['CHN']
# 中国、美国子集
df_china_us = pwt.loc[['CHN','USA']]
# 变量子集
df_sub_china_us = pwt.loc[['CHN', 'USA']][['rgdpe','rgdpo']]

如果需要选择某一年的截面数据：


In [ ]:
pwt.loc[(slice(None), [2019]), :]
# 1992年之后的数据
pwt.loc[(slice(None), slice(1992, None)), :]

这里使用了`df.loc`结合`slice`函数的方法，注意：

- `slice(None)`: 这表示选择 所有 `countrycode`。
- `slice(1992, None)`: 这表示从 `year` 的 1992年 开始，选择到 **所有**后续年份。由于索引是排序的（通常情况下），这有效地选择了所有 `year > 1992` 的数据。
- `:`表示选择所有列。

上面的例子使用`slice`函数不是那么直观，也可以使用`df.index.get_level_values('year')`提取索引`year`的值，形成一个序列（可以另存为一个变量），然后利用表达式生成一个布尔序列，对数据框进行筛选：


In [ ]:
pwt[pwt.index.get_level_values('year') > 1992]

当然，可以同时选择指定的变量和年份，例如：


In [ ]:
pwt.loc[(slice(None),[2016,2019]), ['rgdpe','rgdpo']]
#
pwt.loc[((["CHN", "USA"], [2016,2019])), ['rgdpe','rgdpo']]

除了通常的排序以外，由于有了二级索引，如果按索引排序，两级索引变量是同时排序的：


In [ ]:
pwt.sort_index()

可以对两级索引以列表的形式分别设定排序的顺序。例如，先将国家代码按字母升序，然后将年降序：


In [ ]:
pwt.sort_index(ascending=[True, False])

### `stack` 和 `unstack`

数据有“长（long）”和“宽（wide）”两种组织方式，Penn World Table 是以“长”的形式保存的。有时候需要在两种数据格式之间进行转换，就需要用到`df.stack()`和`df.unstack()`函数。

注意，`df.unstack()`函数的参数`level=`，设置为哪一级索引，便生成为列。默认在最后一级索引上转换，即年，因此列便为年，行为国家，反之，列为国家，行为年。如下面例子所示，为了简便只保留了三个国家5年的数据：


In [ ]:
pwt_sub = pwt.loc[(["CHN", "KOR", "USA"], slice(2015, None)), ["rgdpe", "pop"]]
# 
pwt_sub_wide = pwt_sub.unstack(level=-1)
# pwt_sub.unstack(level=0)

要获得长格式的数据，使用`df.stack()`即可：


In [ ]:
pwt_sub_wide.stack(future_stack=True)

当我们从一些数据库下载数据时，常见形式为列为不同时期相同变量的值。例如，从世界银行下载人均GDP和人口数据：


In [ ]:
import wbgapi as wb
df = wb.data.DataFrame(series=['NY.GDP.PCAP.CD', "SP.POP.TOTL"], 
                                #time=range(2017,2020),
                                time=['YR2017','YR2018','YR2019'],
                                 numericTimeKeys=True)
df.head()

下载的数据`df`索引是“economy - series”，每一年数据一列。我们希望序列成为列变量，时间成为索引。我们可以先对数据进行转置成宽格式的数据，然后再在国家层面堆叠，使其成为索引，再交换索引排序得到通常的情况：


In [ ]:
df.T.stack(level=0, future_stack=True).swaplevel().sort_index()

另外，stack不是唯一的方法，也可以使用`df.melt()`结合`df.pivot_table()`函数来实现：


In [ ]:
df_reset = df.reset_index()
df_long = df_reset.melt(id_vars=['economy', 'series'], var_name='year', value_name='value')
df_long.pivot_table(index=['economy', 'year'], columns='series', values='value')

### Pandas中的分组计算（`groupby`）

Pandas 的分组（`groupby()`）方法按照“分割-应用-组合（split-apply-combine）”的原理，创建一个 groupby 对象，可以应用各种方法来聚合、转换或过滤数据。更多介绍参见Pandas官方文档[Group by: split-apply-combine](https://pandas.pydata.org/docs/user_guide/groupby.html)。

选择合适的方法：

- 如果你的操作只是简单的统计（如求和、平均值），优先使用聚合方法，它们通常效率最高。
- 如果需要返回与原始 DataFrame 相同长度的结果，例如进行组内标准化，使用转换方法。
- 如果需要根据组的属性来决定保留或丢弃整个组，使用过滤方法。
- 当以上方法都无法满足需求时，或者需要执行更复杂的自定义逻辑时，使用**apply()**方法。

#### 聚合方法（Aggregation Methods）

聚合方法将每个组的数据压缩成一个单一的值，是最常用的`groupby`操作，例如`mean()`,`sum()`,`count()`,`size()`,`min()`,`max()`,`std()`,`var()`,`median()`等常见的统计量，或者`first()`,`last()`,`nth(n)`等获取第一个、最好一个或第n个值：


**索引**

例如，根据索引计算世界人口，先在索引上分组，然后使用`.sum()`函数：


In [ ]:
pwt.groupby(level=1)['pop'].sum()

`avh`变量度量了“Average annual hours worked by persons engaged”,让我们分组计算平均，得到按年和按国家平均


In [ ]:
avh = pwt[pwt['avh'].notna()]
fig, ax = plt.subplots(2, 1, figsize=(12, 12))
avh.groupby(level=1)['avh'].mean().sort_values(ascending=False).plot(kind='line', ax=ax[0])
ax[0].set_xlabel("")
ax[0].set_ylabel("Average annual hours worked by persons engaged")
avh.groupby(level=0)['avh'].mean().sort_values(ascending=False)[:25].plot(kind='bar', ax=ax[1])
ax[1].set_xlabel("")
ax[1].set_ylabel("Average annual hours worked by persons engaged")
plt.show()

最常见的是按变量进行分组，例如，按国家名`country`分组，最后一个观测值：


In [ ]:
pwt.groupby(by=['country']).last()

#### 转换方法（Transformation Methods）

- `transform(func)`: 对每个组应用函数，并将结果广播回原始 DataFrame 的形状。
- `rank(method='average')`: 计算组内排名。
- `fillna(value)`: 在组内填充缺失值。


In [ ]:
avh.groupby(level=1)['avh'].transform('mean')
avh.groupby(level=1)['avh'].mean()

注意，转换与聚合的区别,转换将生成的值与原数据观测值一样多，这里是3492个，而聚合只有70个。

`.transform()`方法可以与`lambda`函数相结合，例如：


In [ ]:
pwt.select_dtypes(np.number).groupby(level=0).transform(lambda x: (x - x.mean())/x.std())

#### 过滤方法（Filtration Methods）

过滤方法会根据每个组的某个条件来排除整个组。

- filter(func): 根据一个返回布尔值的函数来过滤组。如果函数对一个组返回 True，则保留该组；否则，删除该组。


In [ ]:
pwt.groupby(level=0).filter(lambda x: x['pop'].mean() > 50)

#### 应用方法（Application Methods）
apply() 方法是最通用的方法，它允许你对每个组应用任何自定义函数。这个函数可以执行聚合、转换或过滤操作，或者任何更复杂的逻辑。

- apply(func): 将一个自定义函数应用于每个组。函数的返回值可以是 Series、DataFrame 或标量。
